In [38]:
from keras.applications.vgg19 import VGG19
from keras.applications.nasnet import NASNetLarge
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
import numpy as np
import pickle
from os import listdir
from os.path import isfile, join
import os
import json
from glob import glob
from tqdm import tqdm_notebook as tqdm
model = VGG19(weights='imagenet', include_top=False)
#model1 = NASNetLarge(weights='imagenet', include_top=False)

In [31]:
'''classes_folders = [x[0] for x in os.walk("./mnist")][1:]
onlyfiles=[]
data={}
try:
    with open('data.pkl', 'rb') as f:
        data= pickle.load(f)
    print("Using previous training data, delete data.pkl to retrain network")
except:
    print("Training new weights")
    for i in range(len(classes_folders)):
        print(i)
        data[i]={}
        data[i]["class"]=classes_folders[i].replace('./mnist/','')
        images=(glob(classes_folders[i]+"/*.jpg"))
        data[i]["images"]=images
        
        for j in tqdm(range(len(images))):
            img = image.load_img(images[j], target_size=(224, 224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            features =+ model.predict(x)
            print(j)
        features=features/len(images)
        data[i]["features"]=features
    with open('data.pkl', 'wb') as f:
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)'''



'classes_folders = [x[0] for x in os.walk("./mnist")][1:]\nonlyfiles=[]\ndata={}\ntry:\n    with open(\'data.pkl\', \'rb\') as f:\n        data= pickle.load(f)\n    print("Using previous training data, delete data.pkl to retrain network")\nexcept:\n    print("Training new weights")\n    for i in range(len(classes_folders)):\n        print(i)\n        data[i]={}\n        data[i]["class"]=classes_folders[i].replace(\'./mnist/\',\'\')\n        images=(glob(classes_folders[i]+"/*.jpg"))\n        data[i]["images"]=images\n        \n        for j in tqdm(range(len(images))):\n            img = image.load_img(images[j], target_size=(224, 224))\n            x = image.img_to_array(img)\n            x = np.expand_dims(x, axis=0)\n            x = preprocess_input(x)\n            features =+ model.predict(x)\n            print(j)\n        features=features/len(images)\n        data[i]["features"]=features\n    with open(\'data.pkl\', \'wb\') as f:\n        pickle.dump(data, f, pickle.HIGHEST_PROTO

In [40]:
classes_folders = [x[0] for x in os.walk("./data")][1:]
onlyfiles=[]
data={}
try:
    with open('data.pkl', 'rb') as f:
        data= pickle.load(f)
    print("Using previous training data, delete data.pkl to retrain network")
except:
    print("Training new weights")
    for i in range(len(classes_folders)):
        data[i]={}
        data[i]["class"]=classes_folders[i].replace('./data/','')
        images=(glob(classes_folders[i]+"/*.jpg"))
        images.extend(glob(classes_folders[i]+"/*.JPG"))
        images.extend(glob(classes_folders[i]+"/*.jpeg"))
        images.extend(glob(classes_folders[i]+"/*.JPEG"))
        images.extend(glob(classes_folders[i]+"/*.png"))
        images.extend(glob(classes_folders[i]+"/*.PNG"))
        data[i]["images"]=images
        features=[]
        print("Training:"+data[i]["class"])
        for j in tqdm(range(len(images))):
            img = image.load_img(images[j], target_size=(224, 224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            features.append(model.predict(x))
        data[i]["features"]=features
    with open('data.pkl', 'wb') as f:
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

Training new weights
0
Training:18651 Cross Axle 2M with Snap friction


1
Training:6632 Technic Lever 3M


2
Training:3024 Plate 1x1


3
Training:3713 Bush for Cross Axle


4
Training:3040 Roof Tile 1x2x45deg


...

11
Training:2357 Brick corner 1x2x2


12
Training:32123 half Bush


13
Training:3022 Plate 2x2


14
Training:3005 Brick 1x1


15
Training:3069 Flat Tile 1x2


In [41]:
def analy(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    test_features = model.predict(x)
    distance={}
    closest=np.inf
    best_guess=""
    for k in range(len(classes_folders)):
        distance[k]={}
        distance[k]["class"]=data[k]["class"]
        feats = data[k]["features"]
        distance[k]["distance"]=np.inf
        for m in range(len(feats)):
            if distance[k]["distance"]>np.abs(np.linalg.norm(feats[m]-test_features)):
                distance[k]["distance"]=np.abs(np.linalg.norm(feats[m]-test_features))
        if distance[k]["distance"]<closest:
            best_guess = distance[k]["class"]
    sorted(distance, key=lambda x: (distance[x]['distance']))
    Y=[x['distance'] for x in distance.values()]
    X = [x['class'] for x in distance.values()]
    best_guesses= [x for _,x in sorted(zip(Y,X))]
    best_distances= [x for _,x in sorted(zip(X,Y))]
    return best_guesses[0]
    #print("This is most likely: "+best_guesses[0])
   # for l in range(len(best_guesses)):
       # print("#{}: {}".format(l+1,best_guesses[l]))

In [42]:
analy("test1.jpg")

'6632 Technic Lever 3M'

In [44]:
correct=0
total=0
valid_folders = [x[0] for x in os.walk("./valid")][1:]
for i in range(len(valid_folders)):
    valimages=(glob(valid_folders[i]+"/*.jpg"))
    valimages.extend(glob(valid_folders[i]+"/*.JPG"))
    valimages.extend(glob(valid_folders[i]+"/*.jpeg"))
    valimages.extend(glob(valid_folders[i]+"/*.JPEG"))
    valimages.extend(glob(valid_folders[i]+"/*.png"))
    valimages.extend(glob(valid_folders[i]+"/*.PNG"))
    for j in tqdm(range(len(valimages))):
        prediction = analy(valimages[j])
        if prediction in valimages[j]:
            correct=correct+1.0
        total=total+1.0
        accuracy=correct/total
print(accuracy)
        

1.0
